In [1]:
# Revove Unwanted warning
import warnings
warnings.simplefilter(action='ignore', category = FutureWarning)

# Data extraction and management
import pandas as pd
import numpy as np
from pandas_datareader.data import DataReader
from pandas_datareader import data as pdr
from pandas_datareader.nasdaq_trader import get_nasdaq_symbols
import yfinance as yf

#Feature Engineering
from sklearn.preprocessing import StandardScaler

# Machine Learning
from sklearn.cluster import KMeans
from sklearn import metrics
from kneed import KneeLocator

# Cointegration and Statistics
from statsmodels.tsa.stattools import coint
import statsmodels.api as sm

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

In [6]:
# Data Extraction
# Set Data Extraction parameters
start_date = "2017-01-01"
end_date = "2022-06-01"
file_name = "data/raw_data_etf.csv"
file_name_coint = "data/raw_data_coint_pairs.csv"
load_existing = False
load_coint_pairs = False

In [9]:
# Get New or Load existing Data
# Allow 15 mins for new data
if not load_existing:
    symbols = get_nasdaq_symbols()
    symbols = symbols[(symbols["ETF"] == True) & (symbols["Market Category"] == "G")] # G = NASDAQ GLOBAL MARKET
    symbols = list(symbols.index.values)
    # data    = 
    yf.pdr_override()
    # df = DataReader(name=symbol, data_source='yahoo', start=start_date, end=end_date)
    data = pdr.get_data_yahoo(symbols, start=start_date, end=end_date)["Adj Close"]
    data
    data.to_csv(file_name)
else:
    data = pd.read_csv(file_name)
# symbols 

[*********************100%***********************]  543 of 543 completed


97 Failed downloads:
['INFR', 'RNEW', 'UPWD', 'SDSI', 'BABX', 'PYPT', 'RUNN', 'BIGB', 'CAFG', 'STXK', 'FDIF', 'BKWO', 'BRNY', 'FDCF', 'UTRE', 'AMZD', 'FBL', 'AMZU', 'TWEB', 'MSFD', 'DYTA', 'PTEC', 'CCSO', 'QYLE', 'FTGS', 'MCSE', 'ERET', 'STXD', 'LITP', 'DVAL', 'TSLL', 'BKIV', 'COWG', 'MKAM', 'MEDX', 'UTWO', 'CLOA', 'AAPB', 'SPAQ', 'BDGS', 'BLLD', 'DIVD', 'PRVT', 'FBOT', 'TBIL', 'IBTM', 'FDTX', 'USCL', 'AAPD', 'EFRA', 'NIKL', 'TSL', 'UFIV', 'MODL', 'BSCW', 'USVN', 'OBIL', 'GGLL', 'TSLQ', 'CONL', 'FIXT', 'UTEN', 'AMID', 'UNIY', 'ILIT', 'STXG', 'BSVO', 'SETM', 'FDFF', 'LZRD', 'UTWY', 'COPJ', 'NVDL', 'BSJU', 'MAXI', 'ETEC', 'XBIL', 'MSFU', 'CIRC', 'UTHY', 'FMED', 'TSLS', 'URNJ', 'GGLS', 'NTZG', 'NVDS', 'IWTR', 'ICOP', 'STXV', 'AOTG', 'PDBA', 'BIGT', 'IBOT', 'VFLO', 'QQQS', 'AAPU', 'BSMW']: Exception("%ticker%: Data doesn't exist for startDate = 1483246800, endDate = 1654056000")


In [14]:
data.dropna(axis=1, inplace=True)
# data = data.set_index("Date")
# data.isnull().values.any()

False

In [12]:
data

,AADR,AAXJ,ACWI,ACWX,AGNG,AGZD,AIA,AIRR,ALTY,ANGL,...,VTHR,VTIP,VTWG,VTWO,VTWV,VWOB,VXUS,VYMI,WOOD,XT
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-03,38.477360,49.435772,52.544994,34.084599,14.656332,38.904072,41.399559,23.497471,9.050782,20.983917,...,93.251923,40.912411,107.183708,50.035992,92.376923,57.906582,38.256912,43.739094,47.666115,25.155079
2017-01-04,39.139633,49.925941,52.994400,34.463394,14.656332,38.992863,41.620762,23.844021,9.177998,21.049280,...,94.323669,40.929058,109.434082,50.786449,93.774551,58.137321,38.679142,44.182594,48.091228,25.482019
2017-01-05,39.559376,50.630009,53.197063,34.758038,14.656332,38.984829,42.133934,23.439217,9.246455,21.100130,...,94.152557,40.978992,108.591385,50.280003,92.510483,58.509468,38.968918,44.610821,48.055805,25.566090
2017-01-06,39.428802,50.380466,53.205883,34.614929,14.656332,38.952496,41.956974,23.439217,9.335508,21.121922,...,94.395729,40.912411,108.035988,50.068233,92.011948,58.345688,38.886116,44.373768,48.055805,25.612785
2017-01-09,39.568710,50.425026,53.082523,34.564419,14.656332,38.855625,42.072014,22.945551,9.275534,21.136442,...,94.071495,40.937382,107.968964,49.732132,90.899185,58.494576,38.811607,44.251427,47.745823,25.675373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-24,53.396790,66.754349,86.258850,46.382626,25.621773,41.882462,59.333374,38.372028,11.192552,26.842985,...,172.742477,47.960266,149.061600,69.456306,123.129395,61.259541,52.907505,60.628033,83.268661,49.225014
2022-05-25,53.723843,67.067253,86.852531,46.430592,25.725248,41.900730,59.797897,38.816185,11.276426,27.127337,...,174.810608,48.016949,152.010941,70.750885,125.593735,61.674484,52.993767,60.796467,84.385849,49.769646
2022-05-26,54.579967,68.142838,88.283203,47.025364,25.828720,41.946411,61.007603,39.474934,11.374278,27.487522,...,178.476364,48.168091,156.009430,72.290649,127.931442,61.844215,53.626350,61.329853,85.686020,50.749989
